In [ ]:
import carla
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)
assert client.get_client_version() == client.get_server_version()

In [ ]:


# print(client.get_server_version())

print(dir(client))
print(client.get_world())
print(client.get_available_maps())

In [ ]:

client.set_timeout(60.0)
world = client.load_world('Town02')
client.set_timeout(2.0)
print(client.get_world())
settings = world.get_settings()
settings.fixed_delta_secodnds = 0.05 #must be less than 0.1, or else physics will be noisy
#must use fixed delta seconds and synchronous mode for python api controlled sim, or else 
#camera and sensor data may not match simulation properly and will be noisy 
settings.synchronous_mode = True 
world.apply_settings(settings)

In [ ]:
import random
actor_list = []
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle')) # lets choose a vehicle at random

# lets choose a random spawn point
transform = random.choice(world.get_map().get_spawn_points()) 

#spawn a vehicle
vehicle = world.spawn_actor(bp, transform) 
actor_list.append(vehicle)

vehicle.set_autopilot(True)

In [ ]:
#lets create waypoints for driving the vehicle around automatically
m= world.get_map()
waypoint = m.get_waypoint(transform.location)

#lets add more vehicles
for _ in range(0, 10):
    transform = random.choice(m.get_spawn_points())

    bp_vehicle = random.choice(blueprint_library.filter('vehicle'))

    # This time we are using try_spawn_actor. If the spot is already
    # occupied by another object, the function will return None.
    other_vehicle = world.try_spawn_actor(bp_vehicle, transform)
    if other_vehicle is not None:
        print(npc)
        other_vehicle.set_autopilot(True)
        actor_list.append(other_vehicle)

In [ ]:
import queue
#example for getting camera image
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
image_queue = queue.Queue()
camera.listen(image_queue.put)
actor_list.append(camera)

#example for getting depth camera image
camera_depth = blueprint_library.find('sensor.camera.depth')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera_d = world.spawn_actor(camera_depth, camera_transform, attach_to=vehicle)
image_queue_depth = queue.Queue()
camera_d.listen(image_queue_depth.put)
actor_list.append(camera_d)

#example for getting semantic segmentation camera image
camera_semseg = blueprint_library.find('sensor.camera.semantic_segmentation')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera_seg = world.spawn_actor(camera_semseg, camera_transform, attach_to=vehicle)
image_queue_seg = queue.Queue()
camera_seg.listen(image_queue_seg.put)
actor_list.append(camera_seg)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
from scipy.spatial import distance

def get_mask(seg_im, rgb_value):
    # rgb_value should be somethiing like np.uint8([[[70, 70, 70]]])
    # seg_im should be in HSV
    
    hsv_value = cv2.cvtColor(rgb_value, cv2.COLOR_RGB2HSV)
    
    hsv_low = np.array([[[hsv_value[0][0][0]-5, hsv_value[0][0][1], hsv_value[0][0][2]-5]]])
    hsv_high = np.array([[[hsv_value[0][0][0]+5, hsv_value[0][0][1], hsv_value[0][0][2]+5]]])
    
    mask = cv2.inRange(seg_im, hsv_low, hsv_high)
    return mask

def get_bbox_from_mask(mask):
    label_mask = measure.label(mask)
    props = measure.regionprops(label_mask)
    
    return [prop.bbox for prop in props]

In [ ]:
world.tick()

In [ ]:
#rgb camera
image = image_queue.get()

#semantic segmentation camera
image_seg  = image_queue_seg.get()

#depth camera
image_depth = image_queue_depth.get()

In [ ]:
from PIL import Image

def carla_image_to_pil(carla_image):
    # Check if the input is a CARLA image
    if not isinstance(carla_image, carla.Image):
        raise ValueError("Input must be a CARLA Image.")

    # Get the image dimensions and format
    width, height = carla_image.width, carla_image.height
    # Convert the CARLA image to a numpy array
    array = np.frombuffer(carla_image.raw_data, dtype=np.dtype("uint8"))
    array = np.reshape(array, (height, width, 4))
    array = array[:, :, (2, 1, 0)]
    return Image.fromarray(array, "RGB")


In [ ]:
display(carla_image_to_pil(image))
display(carla_image_to_pil(image_seg))
display(carla_image_to_pil(image_depth))

In [ ]:
import ipywidgets as widgets
from IPython.display import display
keyboard_input = widgets.Text()

def handle_keypress(change):
    if change['type'] == 'change' and change['name'] == 'value':
        last_key_pressed = change['new'][-1] if change['new'] else ''
        print(f"Key pressed: {last_key_pressed}")
        keyboard_input.value = ''

keyboard_input.observe(handle_keypress)
display(keyboard_input)

In [ ]:
import pygame
import sys
# Set up the screen (window) to capture events
screen = pygame.display.set_mode((400, 300))

print('hello')
# Game loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        
        if event.type == pygame.KEYDOWN:
            print(f"Key pressed: {event.key} ({pygame.key.name(event.key)})")
            world.tick()

    # Update the display
    pygame.display.flip()